<a href="https://colab.research.google.com/github/Sanjaya-Shastry/APU-chatbot/blob/main/APU_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Sanjaya-Shastry/APU-chatbot/refs/heads/main/APU_scraped.txt

--2025-07-29 12:44:52--  https://raw.githubusercontent.com/Sanjaya-Shastry/APU-chatbot/refs/heads/main/APU_scraped.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3054995 (2.9M) [text/plain]
Saving to: ‘APU_scraped.txt’

APU_scraped.txt     100%[===================>]   2.91M  --.-KB/s    in 0.06s   

2025-07-29 12:44:52 (45.4 MB/s) - ‘APU_scraped.txt’ saved [3054995/3054995]



In [3]:
%pip install -q langchain faiss-cpu chromadb sentence-transformers \
                 langchain-openai python-dotenv llama-index-embeddings-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00

In [5]:
%pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00


In [6]:
import os
import uuid
from google.colab import userdata

from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [7]:
with open("APU_scraped.txt", "r", encoding="utf-8") as f:
    text = f.read()

full_text = text

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = [Document(page_content=full_text)]
chunks = splitter.split_documents(docs)

In [13]:
print(f"Total chunks: {len(chunks)}") #checking the number of chunks

Total chunks: 304


In [14]:
api_key = userdata.get("OPEN_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [18]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

In [19]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

In [20]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

In [21]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


/tmp/ipython-input-21-1046491335.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [22]:
retriever = vectorstore.as_retriever()

In [23]:
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever,memory=memory)

In [24]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [25]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [26]:
import gradio as gr

In [27]:
view = gr.ChatInterface(chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98a50c97ec5f90a430.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
